# Automatic differentialtion
## Automatic differentiation with torch.autograd
- 신경망을 학습할 때 **back propagation** 을 주로 사용하며, 손실함수의 **gradient**에 따라 모델의 weigth가 조절됨
- Pytorch에는 `torch.autograd` 라는 내장 미분계산 엔진이 있어 gradient 자동 계산을 지원함
- 입력 백터 `x`, 파라미터 `w`, `b` 그리고 어떤 손실함수로 구성된 단일층 신경망을 고려했을 때, 아래와 같이 Pytorch로 정의할 수 있음

In [42]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph
- 위 코드는 아래의 **computational graph**를 정의함
![Diagram showing a computational graph with two parameters 'w' and 'b' to compute the gradients of loss.](images/computational-graph.png)
- `w`, `b`는 최적화가 필요한 파라미터이므로 손실함수의 gradients의 자동 계산을 위해 `requires_grad` 를 true로 설정하였음
> Tensor를 생성할 때 requires_grad를 설정하거나 만든 뒤에는 x.requires_grad_(True)로 설정 가능함
- computational graph 생성을 위해 실행한 함수는 사실 `Funtion` class의 object임
- 이 object는 forword 방향의 연산과, 미분하는 동안의 backward propagarion의 계산을 어떻게 해야하는지 알고 있으며, Tensor의 `grad_fn` property에 저장하고 있음

In [43]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f5660416f90>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f563cb31310>


미분 계산을 위해 `loss.backward()`을 호출 후 `w.grad`, `b.grad`를 호출하여 값 확인

In [44]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2564, 0.0207, 0.3236],
        [0.2564, 0.0207, 0.3236],
        [0.2564, 0.0207, 0.3236],
        [0.2564, 0.0207, 0.3236],
        [0.2564, 0.0207, 0.3236]])
tensor([0.2564, 0.0207, 0.3236])


> **Note** requires_grad 설정이 True인 leaf node에서만 grad를 사용할 수 있음. 또한 성능을 고려하여 주어진 그래프에서 gradient는 한번만 호출될 수 있음. 같은 그래프에서 반복적으로 호출하기 위해서는 `backward` 호출에 retain_graph=True로 설정해야 함

### Disabling gradient tarcking
모든 Tensor는 기본적으로 `requires_trad=True`로 설정되어 계산 이력이 추적되어, gradient 계산을 지원함. 그러나 학습된 모델에서 입력값을 추론하는 경우에는 단지 forward만 필요할 것임. 이러한 경우 `with torch.no_grad()` 블럭을 사용하여 기본 동작을 정지할 수 있음

In [45]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


`detach` 함수를 사용하여 같은 결과를 얻을 수 있음

In [46]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


gradient tracking을 정지하는 이유는 아래와 같음
- [fine tuning a pre-trained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html) 에서 자주 사용하는 시나리오로써 신경망의 파라미터를 고정하기 위함
- 이미 학습이 끝난 모델을 사용하여 추론하는 경우 forward pass만 필요하므로 gradient 추적을 하지 않음으로써 성능 향상이 가능함

## More on Computational Graph

개념적으로 Autograd는 Function Object로 구성된 DAG 안에 있는 모든 연산자와 데이터를 유지함. DAG에서 leaf는 입력 Tensor를 root는 출력 Tensor이며, Leaf(input)에서 Root(output)까지 추적이 가능하므로, chain rule을 사용하여 자동으로 gradient를 계산할 수 있음.

Forward pass에서 autograd는 두 가지를 동시에 수행함
- 연산자를 실행하여 Tensor의 계산 결과를 제공함
- DAG에서 연산자의 gradient 기능을 유지

Backward pass에서 DAG root에서 `backward()`가 호출되었을 때 autograd는
- 각 `.grad_fn`에서 gradient를 계산
- Chain Rule를 사용하여 각 Tensor의 .grad 속성을 누적 계산하고, leaf tensor까지 모든 방향으로 전파함

### DAGs are dynamic in Pytorch
- 주목할 점은 그래프가 처음부터 다시 생성된다는 것
- `.backward()` 호출 후 autograd는 새 그래프를 생성하기 시작. 이것이 바로 모델에서 제어 흐름을 사용할 수 있도록 함. 필요한 경우 반복할 때 마다 모양, 크기 및 연산의 변경이 가능함

### Optimization pass
- 최적화는 각 학습 단계에서 모델 오차를 줄이기 위해 파라미터를 조절하는 단계
- 모든 최적화 로직은 `optimizer` object로 캡슐레이션 되어있음
- ADAM, RMSProp와 같은 다양한 최적화 알고리즘을 사용할 수 있음 ([different optimizers](https://pytorch.org/docs/stable/optim.html))
- 학습률를 전달하고 학습에 필요한 파라미터를 등록함으로써 optimizer를 초기화 할 수 있음

In [51]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

반복 학습과정 내부에는 최적화가 3단계로 진행됨
- 모델 파라미터의 경사 초기화를 위해 `optimizer.zero_grad()` 호출함. 기본값에 의해 경사는 누적됨. 이중 계산을 막기 위해 각 반복에서 명시적으로 0을 초기화해야 함
- `loss.backwards()`을 사용하여 예측 오차를 역전파. Pytorch는 각 파라미터 손실의 경사를 저장함.
- `optimizer.step()` 호출하면 역전파 경로에서 수집된 경사로 인해 파라미터를 조절할 수 있음.


### Full implementation

In [52]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

`train_loop` 와 `test_loop`에 손실함수, 최적화 함수를 전달. 모델 성능향상을 위해 epoch 수를 높게 조절할 수 있음.

In [53]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300640  [    0/60000]
loss: 2.290309  [ 6400/60000]
loss: 2.286352  [12800/60000]
loss: 2.284961  [19200/60000]
loss: 2.274725  [25600/60000]
loss: 2.273580  [32000/60000]
loss: 2.268600  [38400/60000]
loss: 2.267196  [44800/60000]
loss: 2.237566  [51200/60000]
loss: 2.240211  [57600/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 0.035035 

Epoch 2
-------------------------------
loss: 2.225451  [    0/60000]
loss: 2.211274  [ 6400/60000]
loss: 2.202846  [12800/60000]
loss: 2.202437  [19200/60000]
loss: 2.185632  [25600/60000]
loss: 2.186528  [32000/60000]
loss: 2.178825  [38400/60000]
loss: 2.179127  [44800/60000]
loss: 2.110761  [51200/60000]
loss: 2.115404  [57600/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 0.033081 

Epoch 3
-------------------------------
loss: 2.113491  [    0/60000]
loss: 2.087825  [ 6400/60000]
loss: 2.064968  [12800/60000]
loss: 2.056619  [19200/60000]
loss: 2.046947  [25600/60000]
loss: 2.065896  [32000/600